In [49]:
import numpy as np
import pandas as pd
import math
import pickle
import scipy.sparse as sp
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss



In [93]:
f = open('../data/full_train_yx','rb')
train = pickle.load(f)
n_train = len(train)
print(n_train)
f = open('../data/full_test_yx','rb')
test = pickle.load(f)
n_test = len(test)
print(n_test)

59716
14929


In [51]:
datadir = '../data/'
data1 = pd.read_csv(os.path.join(datadir,'gender_age_train.csv'),
                      index_col='device_id')

data2 = pd.read_csv(os.path.join(datadir,'gender_age_test.csv'),
                     index_col = 'device_id')

data = pd.concat([data1, data2],sort=False)

gatrain = data.loc[train]
gatest = data.loc[test]


In [52]:
gatrain['trainrow'] = np.arange(gatrain.shape[0],dtype=int)
gatest['testrow'] = np.arange(gatest.shape[0],dtype=int)
phone = pd.read_csv(os.path.join(datadir,'phone_brand_device_model.csv'))
# Get rid of duplicate device ids in phone
phone = phone.drop_duplicates('device_id',keep='first').set_index('device_id')
events = pd.read_csv(os.path.join(datadir,'events.csv'),
                     parse_dates=['timestamp'], index_col='event_id')
appevents = pd.read_csv(os.path.join(datadir,'app_events.csv'), 
                        usecols=['event_id','app_id','is_active'],
                        dtype={'is_active':bool})
applabels = pd.read_csv(os.path.join(datadir,'app_labels.csv'))
appencoder = LabelEncoder().fit(appevents.app_id)
appevents['app'] = appencoder.transform(appevents.app_id)
napps = len(appencoder.classes_)

deviceapps = (appevents.merge(events[['device_id']], how='left',left_on='event_id',right_index=True)
                       .groupby(['device_id','app'])['app'].agg(['size'])
                       .merge(gatrain[['trainrow']], how='left', left_index=True, right_index=True)
                       .merge(gatest[['testrow']], how='left', left_index=True, right_index=True)
                       .reset_index())
deviceapps.head()

,device_id,app,size,trainrow,testrow
0,-9222956879900151005,548,18,49974.0,NaN
1,-9222956879900151005,1096,18,49974.0,NaN
2,-9222956879900151005,1248,26,49974.0,NaN
3,-9222956879900151005,1545,12,49974.0,NaN
4,-9222956879900151005,1664,18,49974.0,NaN


In [92]:
d = deviceapps.fillna(0)
test_index = max(d.trainrow) + 1
test_index = int(test_index)
print(test_index)
user = d.trainrow + (d.testrow != 0) * ( test_index + d.testrow)
d['user'] = user
d.user = d.user.astype(int)
d.head()

14926.0
59714.0
59715


,device_id,app,size,trainrow,testrow,user
0,-9222956879900151005,548,18,49974.0,0.0,49974
1,-9222956879900151005,1096,18,49974.0,0.0,49974
2,-9222956879900151005,1248,26,49974.0,0.0,49974
3,-9222956879900151005,1545,12,49974.0,0.0,49974
4,-9222956879900151005,1664,18,49974.0,0.0,49974


In [91]:
N_users = max(d.user) + 1

print(N_users)
adj_matrix = d[['user', 'app','size']].copy()
adj_matrix.head()

23291


,user,app,size
0,49974,548,18
1,49974,1096,18
2,49974,1248,26
3,49974,1545,12
4,49974,1664,18


In [82]:
# write bipartite graph to file
f = open("weights.txt", "w+")
A = adj_matrix.values
N = A.shape[0]
for i in range(N):
    f.write("u"+str(A[i,0])+'\ti'+str(A[i,1])+'\t'+str(A[i,2])+'\n')

2369025


In [115]:
# load embeddings
d = 64 # latent dimension
f_name = "vec_u.dat"

In [116]:
X = np.zeros((n_train+n_test,d))

with open(f_name) as f:
    D = f.readlines()
for line in D:
    vals = np.zeros(d)
    data = line.split(' ')
    index = int(data[0][1:])
    for i in range(d):
        vals[i] = float(data[i+1])
    X[index,:] = vals

Xtr = X[0:n_train,:]
Xte = X[n_train:,:]
print(Xtr.shape)

(59716, 64)


In [69]:
# subset
subset = False
if subset:
    idx = np.sort(idx)
    X_train = X_train[idx,:]

In [70]:
brandencoder = LabelEncoder().fit(phone.phone_brand)
phone['brand'] = brandencoder.transform(phone['phone_brand'])
gatrain['brand'] = phone['brand']
gatest['brand'] = phone['brand']
Xtr_brand = csr_matrix((np.ones(gatrain.shape[0]), 
                       (gatrain.trainrow, gatrain.brand)))
Xte_brand = csr_matrix((np.ones(gatest.shape[0]), 
                       (gatest.testrow, gatest.brand)))
print('Brand features: train shape {}, test shape {}'.format(Xtr_brand.shape, Xte_brand.shape))

Brand features: train shape (59716, 131), test shape (14929, 131)


In [71]:
d = deviceapps.dropna(subset=['trainrow'])
Xtr_app = csr_matrix((np.ones(d.shape[0]), (d.trainrow, d.app)), 
                      shape=(gatrain.shape[0],napps))
d = deviceapps.dropna(subset=['testrow'])
Xte_app = csr_matrix((np.ones(d.shape[0]), (d.testrow, d.app)), 
                      shape=(gatest.shape[0],napps))
print('Apps data: train shape {}, test shape {}'.format(Xtr_app.shape, Xte_app.shape))

Apps data: train shape (59716, 19237), test shape (14929, 19237)


In [72]:
applabels = applabels.loc[applabels.app_id.isin(appevents.app_id.unique())]
applabels['app'] = appencoder.transform(applabels.app_id)
labelencoder = LabelEncoder().fit(applabels.label_id)
applabels['label'] = labelencoder.transform(applabels.label_id)
nlabels = len(labelencoder.classes_)
devicelabels = (deviceapps[['device_id','app']]
                .merge(applabels[['app','label']])
                .groupby(['device_id','label'])['app'].agg(['size'])
                .merge(gatrain[['trainrow']], how='left', left_index=True, right_index=True)
                .merge(gatest[['testrow']], how='left', left_index=True, right_index=True)
                .reset_index())
devicelabels.head()

d = devicelabels.dropna(subset=['trainrow'])
Xtr_label = csr_matrix((np.ones(d.shape[0]), (d.trainrow, d.label)), 
                      shape=(gatrain.shape[0],nlabels))
d = devicelabels.dropna(subset=['testrow'])
Xte_label = csr_matrix((np.ones(d.shape[0]), (d.testrow, d.label)), 
                      shape=(gatest.shape[0],nlabels))
print('Labels data: train shape {}, test shape {}'.format(Xtr_label.shape, Xte_label.shape))

Labels data: train shape (59716, 492), test shape (14929, 492)


In [73]:
m = phone.phone_brand.str.cat(phone.device_model)
modelencoder = LabelEncoder().fit(m)
phone['model'] = modelencoder.transform(m)
gatrain['model'] = phone['model']
gatest['model'] = phone['model']
Xtr_model = csr_matrix((np.ones(gatrain.shape[0]), 
                       (gatrain.trainrow, gatrain.model)))
Xte_model = csr_matrix((np.ones(gatest.shape[0]), 
                       (gatest.testrow, gatest.model)))
print('Model features: train shape {}, test shape {}'.format(Xtr_model.shape, Xte_model.shape))

Model features: train shape (59716, 1667), test shape (14929, 1667)


In [74]:
labels = gatrain[['trainrow','group']].copy()
labels.group = labels.group.astype('category')
labels.group = labels.group.cat.codes
labels.head()

,trainrow,group
device_id,,
3939996495880662904,0,8
-5127195038907890249,1,0
3074841628121636464,2,10
1930589242199599785,3,10
-2845082486521958712,4,10


In [75]:
targetencoder = LabelEncoder().fit(gatrain.group)
y = targetencoder.transform(gatrain.group)
nclasses = len(targetencoder.classes_)

In [76]:
if subset:
    y = y[idx]
    Xtr_brand = Xtr_brand[idx,:]
    Xtr_model = Xtr_model[idx,:]
    Xtr_app = Xtr_app[idx,:]
    Xtr_label = Xtr_label[idx,:]

In [117]:
# concatenate feature vectors
Xtrain = hstack((Xtr_brand, Xtr_model, Xtr_app, Xtr_label, Xtr), format='csr')
Xtest =  hstack((Xte_brand, Xte_model, Xte_app, Xte_label, Xte), format='csr')
print('All features: train shape {}, test shape {}'.format(Xtrain.shape, Xtest.shape))

All features: train shape (59716, 21591), test shape (14929, 21591)


In [99]:
from sklearn.model_selection import StratifiedKFold
def score(clf,n=1, random_state = 0):
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    pred = np.zeros((y.shape[0],nclasses))
    avg_acc = 0
    reg_acc = 0
    for itrain, itest in kf.split(Xtrain, y):
        Xtr, Xte = Xtrain[itrain, :], Xtrain[itest, :]
        ytr, yte = y[itrain], y[itest]
        clf.fit(Xtr, ytr)
        pred[itest,:] = clf.predict_proba(Xte)
        reg_acc += clf.score(Xte,yte)
        best_n = np.argsort(pred[itest,:], axis=1)[:,-n:] # top 3
        if n==1:
            acc = clf.score(Xte,yte)
        else:
            acc = 0
            m = best_n.shape[0]
            for i in range(m):
                if yte[i] in list(best_n[i,:]):
                    acc += 1
            acc = acc / np.float(m)
        avg_acc += acc
    avg_acc = avg_acc/np.float(5)
    reg_acc = reg_acc/np.float(5)
    return (reg_acc, avg_acc), log_loss(y, pred)

In [101]:
Cs = [0.001,0.01,0.1,1]
for c in Cs:
    print(c)
    clf = LogisticRegression(C=c,multi_class='multinomial',solver='newton-cg',max_iter=1000)
    acc, loss = score(clf,3)
    print("Accuracy:",acc[0])
    print("Accuracy@3",acc[1])

0.001
Accuracy: 0.1780772677130522
Accuracy@3 0.43361918574333763
0.01
Accuracy: 0.1942370349504019
Accuracy@3 0.4613509648120225
0.1
Accuracy: 0.19423630308359846
Accuracy@3 0.4598108322436166
1
Accuracy: 0.18283277860312122
Accuracy@3 0.44117171825794477


In [118]:
# Test
targetencoder = LabelEncoder().fit(gatest.group)
y_test = targetencoder.transform(gatest.group)

c = 0.01
clf = LogisticRegression(C=c,multi_class='multinomial',solver='newton-cg',max_iter=1000).fit(Xtrain,y)
acc = clf.score(Xtest,y_test)
print("Accuracy", acc)

n = 3 # top 3
pred= clf.predict_proba(Xtest)
best_n = np.argsort(pred, axis=1)[:,-n:] # top 3
acc = 0
m = best_n.shape[0]
for i in range(m):
    if y_test[i] in list(best_n[i,:]):
        acc += 1
acc = acc / np.float(m)
print("Accuracy@3", acc)

Accuracy 0.19753499899524415
Accuracy@3 0.46252260700649744
